In [2]:
import pandas as pd
import glob
import geopandas as gpd
import hdbscan
import matplotlib.pyplot as plt
from pyproj import Transformer
import folium
import re
import numpy as np


### Samenvoegen van alle meetputen 

In [3]:
# Alle meetpunten inladen vanuit subfolders van Meetpunten
csv_files = glob.glob('Meetpunten/*/*.csv')  # zoekt in alle subfolders

# DataFrames inlezen en samenvoegen
df_list = []
for file in csv_files:
    df = pd.read_csv(file, sep=';', encoding='utf-8')  # gebruik ';' als delimiter
    df_list.append(df)

# Samenvoegen in één DataFrame
df_all = pd.concat(df_list, ignore_index=True)
print(f"Totaal aantal meetpunten: {len(df_all)}")

# Exporteren naar CSV
df_all.to_csv('all_meetpunten.csv', index=False)
print("Alle meetpunten opgeslagen in 'all_meetpunten.csv'")


Totaal aantal meetpunten: 3342
Alle meetpunten opgeslagen in 'all_meetpunten.csv'


### Clusters maken

In [4]:
# 2. Clustering met HDBSCAN op RD-coördinaten (meters)
# Zorg dat de kolommen 'GeometriePuntX_RD' en 'GeometriePuntY_RD' bestaan
df_all = df_all.dropna(subset=['GeometriePuntX_RD', 'GeometriePuntY_RD'])
coords = df_all[['GeometriePuntX_RD', 'GeometriePuntY_RD']].values

# HDBSCAN-instantie: pas min_cluster_size en min_samples aan naar behoefte
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=10,
    min_samples=1,
    metric='euclidean'
)
labels = clusterer.fit_predict(coords)

# 3. Resultaten toevoegen aan DataFrame
df_all['cluster_id'] = labels
# Optioneel: waarschijnlijkheidscore per punt
df_all['cluster_prob'] = clusterer.probabilities_

# 4. Opslaan van de CSV met clusters
df_all.to_csv('meetpunten_met_clusters.csv', index=False)
print("CSV 'meetpunten_met_clusters.csv' opgeslagen met cluster labels.")


CSV 'meetpunten_met_clusters.csv' opgeslagen met cluster labels.


### HTML kaartje maken 

In [5]:
# 1. Inladen van de CSV met clusters
df_all = pd.read_csv('meetpunten_met_clusters.csv')

# 2. Omzetten van RD-coördinaten naar WGS84 (lon, lat)
transformer = Transformer.from_crs('EPSG:28992', 'EPSG:4326', always_xy=True)
df_all['lon'], df_all['lat'] = transformer.transform(
    df_all['GeometriePuntX_RD'].values,
    df_all['GeometriePuntY_RD'].values
)

# 3. Kleuren toewijzen per cluster_id
def get_cluster_colors(cluster_ids):
    # Voorbeeld kleurenpalet
    palette = [
        'red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred',
        'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple',
        'pink', 'lightblue', 'lightgreen', 'gray'
    ]
    return {cid: palette[i % len(palette)] for i, cid in enumerate(cluster_ids)}

cluster_ids = sorted(df_all['cluster_id'].unique())
color_map = get_cluster_colors(cluster_ids)

# 4. Simpele Folium-kaart maken
def create_cluster_map(df, color_map, zoom_start=10):
    center = [df['lat'].mean(), df['lon'].mean()]
    m = folium.Map(location=center, zoom_start=zoom_start)
    for _, row in df.iterrows():
        color = color_map.get(row['cluster_id'], 'gray')
        folium.CircleMarker(
            location=(row['lat'], row['lon']),
            radius=4,
            color=color,
            fill=True,
            fill_opacity=0.7,
            popup=f"Cluster: {row['cluster_id']}"
        ).add_to(m)
    return m

# 5. Maak kaart en sla op als HTML
m = create_cluster_map(df_all, color_map)
output_file = 'clusters_map_simple.html'
m.save(output_file)
print(f"Eenvoudige interactieve kaart met gekleurde clusters opgeslagen als '{output_file}'")


Eenvoudige interactieve kaart met gekleurde clusters opgeslagen als 'clusters_map_simple.html'


### Relevantie van clusters

In [6]:
# 1. Inladen van bestaand CSV-bestand met cluster-id's
df = pd.read_csv('meetpunten_met_clusters.csv')

# 2. Bepaal de set van alle jaren in de dataset
all_years = set(df['Meetjaar'].dropna().unique())

# 3. Filter noise-cluster (cluster_id = -1) uit voor relevantiebepaling
df_non_noise = df[df['cluster_id'] != -1]

# 4. Groepeer per cluster en bepaal unieke jaren per cluster
cluster_years = (
    df_non_noise.groupby('cluster_id')['Meetjaar']
    .apply(lambda yrs: set(yrs.dropna().unique()))
)

# 5. Identificeer relevante clusters (die elk jaar in all_years voorkomen)
relevant_clusters = [
    cid for cid, yrs in cluster_years.items() if yrs == all_years
]
print(f"Relevante clusters: {relevant_clusters}")

# 6. Voeg een nieuwe kolom 'relevant' toe aan de DataFrame
#    True als het meetpunt behoort tot een relevante cluster, anders False
df['relevant'] = df['cluster_id'].isin(relevant_clusters)

# 7. Opslaan van het nieuwe CSV-bestand
df.to_csv('meetpunten_met_clusters_relevant.csv', index=False)
print("meetpunten_met_clusters_relevant.csv opgeslagen met relevante kolom.")


Relevante clusters: [3, 5, 19, 25, 26, 32, 33, 44, 62, 64, 73, 74, 87, 88, 89, 91, 95, 96, 108, 113, 114, 124]
meetpunten_met_clusters_relevant.csv opgeslagen met relevante kolom.


### Samenvoegen meetwaarden en meetpunten clusters

In [7]:
# -------------------------
# 1. Inladen van meetpunten met clusters en relevantie
# -------------------------
df_clusters = pd.read_csv('meetpunten_met_clusters_relevant.csv')
# Unieke mapping per MeetobjectCode
# Alleen meetpunten in relevante clusters (relevant == True) behouden
df_clusters_relevant = (
    df_clusters[df_clusters['relevant']]
    [['MeetobjectCode', 'cluster_id']]
    .drop_duplicates(subset=['MeetobjectCode'])
)

# -------------------------
# 2. Inladen en samenvoegen van meetwaarden (2016–2024)
# -------------------------
meetwaarden_files = glob.glob('Meetwaarden/Waterschap Brabantse Delta/WKP_Meetwaarden_Waterschap Brabantse Delta_*.csv')
waarden_list = []
for file in meetwaarden_files:
    # Jaar uit bestandsnaam halen (bv. '_2016_')
    match = re.search(r'_(\d{4})_', file)
    jaar = int(match.group(1)) if match else None
    df = pd.read_csv(file, sep=';', encoding='utf-8')
    df['year'] = jaar
    waarden_list.append(df)
# Samenvoegen van alle meetwaarden
df_waarden = pd.concat(waarden_list, ignore_index=True)
print(f"Totaal aantal meetwaardenrijen: {len(df_waarden)}")

# -------------------------
# 3. Merge meetwaarden met alleen relevante clusters
df_merged = df_waarden.merge(
    df_clusters_relevant,
    on='MeetobjectCode',
    how='inner'  # alleen waarden met relevante meetpunten behouden
)
print(f"Aantal rijen na inner merge op relevante clusters: {len(df_merged)}")

# -------------------------
# 4. Bepalen van relevante stoffen per cluster
# -------------------------
# Set van alle jaren van interesse in de data
total_years = set(df_merged['year'].dropna().unique())
print(f"Jaren in dataset: {sorted(total_years)}")

# Groepeer per cluster en stof en verzamel gemeten jaren
grouped = (
    df_merged
    .groupby(['cluster_id', 'ParameterOmschrijving'])['year']
    .apply(lambda yrs: set(yrs.dropna().unique()))
    .reset_index(name='years_measured')
)
# Markeer stoffen die in elk jaar van hun cluster zijn gemeten
grouped['relevant_stof'] = grouped['years_measured'].apply(
    lambda s: s == total_years
)

# Filter op relevante stoffen per cluster
grouped_relevant = grouped[grouped['relevant_stof']]
print(f"Aantal relevante cluster-stof combinaties: {len(grouped_relevant)}")

# -------------------------
# 5. Bepalen van globale relevantie over alle clusters
# -------------------------
df_counts = (
    grouped_relevant
    .groupby('ParameterOmschrijving')['cluster_id']
    .nunique()
    .reset_index(name='count_clusters')
    .sort_values('count_clusters', ascending=False)
)
print("Top 10 stoffen met hoogste cluster frequentie:")
print(df_counts.head(40))

# Optioneel: bepalen hoeveel clusters er analyseerbaar zijn
total_clusters = df_clusters_relevant['cluster_id'].nunique()
print(f"Totaal aantal relevante clusters: {total_clusters}")

# Stoffen die in alle relevante clusters voorkomen (indien gewenst)
top_all = df_counts[df_counts['count_clusters'] == total_clusters]
print(f"Stoffen in alle {total_clusters} relevante clusters: {top_all['ParameterOmschrijving'].tolist()}")

# -------------------------
# 6. Opslaan van frequentietabel naar CSV
# -------------------------
# Schrijft alle stoffen met hun aantal relevante clusters naar een CSV-bestand
output_path = 'stoffen_cluster_counts.csv'
df_counts.to_csv(output_path, index=False)
print(f"CSV '{output_path}' opgeslagen met frequentie van relevante stoffen in clusters.")



/var/folders/mg/rgpdn71s30sccy9pj35kkd6c0000gn/T/ipykernel_27858/4140453548.py:22: DtypeWarning: Columns (4,7,8,9,15,16,17,25,26,34,35,40,41,42,43,48,49,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';', encoding='utf-8')
/var/folders/mg/rgpdn71s30sccy9pj35kkd6c0000gn/T/ipykernel_27858/4140453548.py:22: DtypeWarning: Columns (4,7,8,9,16,17,22,34,35,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';', encoding='utf-8')
/var/folders/mg/rgpdn71s30sccy9pj35kkd6c0000gn/T/ipykernel_27858/4140453548.py:22: DtypeWarning: Columns (4,7,8,20,24,25,26,34,35,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';', encoding='utf-8')
/var/folders/mg/rgpdn71s30sccy9pj35kkd6c0000gn/T/ipykernel_27858/4140453548.py:22: DtypeWarning: Columns (4,7,8,20,24,25,26,34,35,42,43) have mixed types. Specify dtype option on import 

Totaal aantal meetwaardenrijen: 1115673
Aantal rijen na inner merge op relevante clusters: 276699
Jaren in dataset: [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Aantal relevante cluster-stof combinaties: 497
Top 10 stoffen met hoogste cluster frequentie:
                                ParameterOmschrijving  count_clusters
18                                        chlorofyl-a              17
48                                           zuurstof              12
2                                                Olie              12
3                                              Schuim              12
4                                                Vuil              12
37                                            nitraat              10
29                                 koolstof organisch              10
30                                              koper              10
31                                               kwik              10
32                                 

In [53]:
import pandas as pd
import numpy as np
from pyproj import Transformer

# 1) Inladen relevante clusters
df_clusters = pd.read_csv('meetpunten_met_clusters_relevant.csv')
df_clusters = df_clusters[df_clusters['relevant']].drop_duplicates('cluster_id')

# 2) Inladen voedselbossen en coördinaten omzetten
df_forest = pd.read_excel('Book4.xlsx')
# Excel-coördinaten zijn RD-meters * 1e6, dus schalen
df_forest['lon_deg'] = df_forest['Lon'] / 1e6
df_forest['lat_deg'] = df_forest['Lat'] / 1e6

# Projectie WGS84 → RD New
transformer = Transformer.from_crs('EPSG:4326', 'EPSG:28992', always_xy=True)
df_forest['x_rd'], df_forest['y_rd'] = transformer.transform(
    df_forest['lon_deg'].values,
    df_forest['lat_deg'].values
)

# 3) Voor ieder cluster het dichtstbijzijnde bos binnen 2 km bepalen
results = []
for _, cl in df_clusters.iterrows():
    x0, y0 = cl['GeometriePuntX_RD'], cl['GeometriePuntY_RD']
    d2 = (df_forest['x_rd'] - x0)**2 + (df_forest['y_rd'] - y0)**2
    idx = d2.idxmin()
    dist = np.sqrt(d2.loc[idx])
    if dist <= 2000:
        results.append({
            'cluster_id': cl['cluster_id'],
            'Naam': df_forest.at[idx, 'Naam'],
            'Grootte': df_forest.at[idx, 'Grootte'],
            'Eerste aanplant': df_forest.at[idx, 'Eerste aanplant'],
            'distance_m': dist
        })

df_forest_map = pd.DataFrame(results)

# 4) Merge met je meetpunten_per_cluster file
df_expanded = pd.merge(
    pd.read_csv('meetpunten_met_clusters_relevant.csv'),
    df_forest_map,
    on='cluster_id',
    how='left'
)

# 5) Opslaan
df_expanded.to_csv(
    'meetpunten_met_clusters_relevant_met_bos.csv',
    index=False
)
print("Nieuw bestand weggeschreven: 'meetpunten_met_clusters_relevant_met_bos.csv'")


Nieuw bestand weggeschreven: 'meetpunten_met_clusters_relevant_met_bos.csv'


In [9]:
import pandas as pd
import glob
import re

# -------------------------
# 1. Define columns to drop early
# -------------------------
drop_cols = [
    'OrgaanCode','OrgaanOmschrijving','OrganismeNaam','OrganismeNaam_Nederlands',
    'BemonsteringsapparaatCode','BemonsteringsapparaatOmschrijving','TyperingCode',
    'TyperingOmschrijving','GrootheidCode','GrootheidOmschrijving','ParameterCode',
    'ParameterCASnummer','BiotaxonNaam','BiotaxonNaam_Nederlands','EenheidCode',
    'EenheidOmschrijving','HoedanigheidCode','HoedanigheidOmschrijving',
    'LevensstadiumCode','LevensstadiumOmschrijving','LengteklasseCode',
    'LengteklasseOmschrijving','GeslachtCode','GeslachtOmschrijving',
    'VerschijningsvormCode','VerschijningsvormOmschrijving','LevensvormCode',
    'LevensvormOmschrijving','GedragCode','GedragOmschrijving','AnalyseCompartimentCode',
    'AnalyseCompartimentOmschrijving','WaardebewerkingsmethodeCode',
    'WaardebewerkingsmethodeOmschrijving','WaardebepalingsmethodeCode',
    'WaardebepalingsmethodeOmschrijving','LocatieTypeWaardeBepalingID',
    'LocatieTypeWaardeBepalingOmschrijving','Alfanumeriekewaarde',
    'KwaliteitsoordeelCode','KwaliteitsoordeelOmschrijving','PublicatieDatumTijd'
]

# -------------------------
# 2. Load and clean meetwaarden into a single DataFrame
# -------------------------
waarden_list = []
for file in glob.glob('Meetwaarden/Waterschap Brabantse Delta/WKP_Meetwaarden_Waterschap Brabantse Delta_*.csv'):
    # Extract year from filename
    jaar = int(re.search(r'_(\d{4})_', file).group(1))
    # Read and drop unwanted columns immediately
    df = pd.read_csv(file, sep=';', encoding='utf-8')
    df.drop(columns=drop_cols, errors='ignore', inplace=True)
    df['year'] = jaar
    waarden_list.append(df)
# Concatenate all years into one DataFrame
df_waarden = pd.concat(waarden_list, ignore_index=True)
print(f"Loaded and cleaned meetwaarden, shape={df_waarden.shape}")

# -------------------------
# 3. Merge meetwaarden with cluster info (keep all meetwaarden rows)
# -------------------------
df_clusters = pd.read_csv('meetpunten_met_clusters_relevant.csv')
# Prepare mapping of MeetobjectCode to cluster_id and relevant flag
df_cluster_map = (
    df_clusters[['MeetobjectCode','cluster_id','relevant']]
    .drop_duplicates(subset=['MeetobjectCode'])
)
# Left join preserves all meetwaarden rows
df_merged = df_waarden.merge(
    df_cluster_map,
    on='MeetobjectCode',
    how='left'
)
print(f"After merging clusters, rows={len(df_merged)} (should equal meetwaarden count)")

# -------------------------
# 4. Merge with forest mapping (one record per cluster)
# -------------------------
df_forest = pd.read_csv('meetpunten_met_clusters_relevant_met_bos.csv')
# Keep nearest forest per cluster_id if multiple
if 'distance_m' in df_forest.columns:
    df_forest_unique = (
        df_forest.sort_values('distance_m')
                 .drop_duplicates(subset=['cluster_id'], keep='first')
    )
else:
    df_forest_unique = df_forest.drop_duplicates(subset=['cluster_id'])
# Merge forest info into df_merged
df_final = df_merged.merge(
    df_forest_unique[['cluster_id','Naam','Grootte','Eerste aanplant','distance_m']],
    on='cluster_id',
    how='left'
)
print(f"After merging forests, rows={len(df_final)} (should still equal meetwaarden count)")

# -------------------------
# 5. df_final now contains:
#      - all original meetwaarde columns minus dropped ones
#      - year, cluster_id, relevant flag
#      - forest info (Naam, Grootte, Eerste aanplant, distance_m)

# -------------------------
# 6. Prepare data for regression
# -------------------------
# Filter to only relevant chemicals (measured in all years within cluster)
# and clusters that have at least one nearby food forest
df_rel = df_final[df_final['relevant']]

# Compute list of chemicals to model
chemicals = df_rel['ParameterOmschrijving'].unique().tolist()

# Aggregate to cluster-year-chemical level
agg_list = []
for chem in chemicals:
    sub = df_rel[df_rel['ParameterOmschrijving'] == chem]
    grp = (
        sub
        .groupby(['cluster_id','year'])
        .agg(
            mean_conc=('Numeriekewaarde','mean'),          # average concentration
            has_forest=('Naam', lambda s: s.notna().any()),
            dist_to_forest=('distance_m','min')   # distance to nearest forest
        )
        .reset_index()
    )
    grp['chemical'] = chem
    agg_list.append(grp)

df_agg = pd.concat(agg_list, ignore_index=True)
print(f"Prepared regression DataFrame with {len(df_agg)} rows")

# -------------------------
# 7. Run regression for a representative chemical
# -------------------------
import statsmodels.formula.api as smf

# Example: run for the first chemical in list
target = chemicals[0]
df_model = df_agg[df_agg['ParameterOmschrijving'] == target]

# OLS with binary forest flag and year fixed effects, clustered SE by cluster
model = smf.ols(
    'mean_conc ~ has_forest + C(year)',
    data=df_model
).fit(
    cov_type='cluster',
    cov_kwds={'groups': df_model['cluster_id']}
)
print(f"Regression results for {target}:")
print(model.summary())

# -------------------------
# 8. (Optional) Loop over all chemicals
# -------------------------
results = []
for chem in chemicals:
    df_m = df_agg[df_agg['ParameterOmschrijving'] == chem]
    if len(df_m) < 10:
        continue
    res = smf.ols('mean_conc ~ has_forest + C(year)', data=df_m).fit(
        cov_type='cluster', cov_kwds={'groups': df_m['cluster_id']}
    )
    coef = res.params.get('has_forest', float('nan'))
    pval = res.pvalues.get('has_forest', float('nan'))
    results.append({'ParameterOmschrijving': chem, 'coef_has_forest': coef, 'pval': pval})
df_results = pd.DataFrame(results)
print("Summary of forest effects across chemicals:")
print(df_results.sort_values('coef_has_forest'))

# End of regression pipeline



/var/folders/mg/rgpdn71s30sccy9pj35kkd6c0000gn/T/ipykernel_27858/1922193198.py:33: DtypeWarning: Columns (4,7,8,9,15,16,17,25,26,34,35,40,41,42,43,48,49,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';', encoding='utf-8')
/var/folders/mg/rgpdn71s30sccy9pj35kkd6c0000gn/T/ipykernel_27858/1922193198.py:33: DtypeWarning: Columns (4,7,8,9,16,17,22,34,35,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';', encoding='utf-8')
/var/folders/mg/rgpdn71s30sccy9pj35kkd6c0000gn/T/ipykernel_27858/1922193198.py:33: DtypeWarning: Columns (4,7,8,20,24,25,26,34,35,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';', encoding='utf-8')
/var/folders/mg/rgpdn71s30sccy9pj35kkd6c0000gn/T/ipykernel_27858/1922193198.py:33: DtypeWarning: Columns (4,7,8,20,24,25,26,34,35,42,43) have mixed types. Specify dtype option on import 

Loaded and cleaned meetwaarden, shape=(1115673, 25)
After merging clusters, rows=1115673 (should equal meetwaarden count)
After merging forests, rows=1115673 (should still equal meetwaarden count)
Prepared regression DataFrame with 30327 rows


ModuleNotFoundError: No module named 'statsmodels'